# Planetary gravity

The aim of this lecture and practical is to investigate  methods for calculating the gravitational 
field for realistic planetary models using **numerical integration** along with  **spherical harmonic** transformations. In particular, 
we will consider a simple model for determining the gravitational field associated with the Earth's ice sheets. 

## Numerical integration

Let's consider  the generic integral
\begin{equation}
I = \int_{a}^{b} f(x) \, \mathrm{d} x. 
\end{equation}
Methods for numerically evaluating such integrals are known as **quadrature rules**, with each approximating
the integral by a finite sum
\begin{equation}
I \approx \sum_{i=0}^{n-1} f(x_{i}) \, w_{i}, 
\end{equation}
where the $x_{0},\dots,x_{n-1}$ are known as **quadrature points** and the $w_{0},\dots,w_{n-1}$ are **quadrature weights**. Different schemes are distinguished by the manner in which these points and weights are chosen. The extension of this idea to integrals in higher dimensions (e.g., surface or volume
integration) is immediate.


Perhaps the simplest quadrature scheme is the **rectangle rule** which is motivated by  definition of an integral as a limit of a sum. 
Here the integration points sub-divide the interval $[a,b]$ in increasing order, though they need not be equally spaced. The $i$th integration weight is chosen such that the corresponding term within the sum gives the area of the rectangle with height $f(x_{i})$ and width $x_{i+1}-x_{i}$, and hence 
\begin{equation}
w_{i} = x_{i+1}-x_{i}.
\end{equation}
As the number of subdivisions is increased, the approximating sum tends to the desired integral precisely because this is how integrals are defined!

A simple code implementing this method is shown below and used to evaluate the integral
\begin{equation}
I = \int_{0}^{\pi} \sin \theta \, \mathrm{d} \theta, 
\end{equation}
using the rectangle rule. The exact value of the integral is 2, as can be readily checked. This allows
us to determine the error on the method for a given number of quadrature points.


In [ ]:
try:
    import pyslfp as sl
except ImportError: 
    %pip install pyslfp --quiet
    import pyslfp as sl

import numpy as np
import matplotlib.pyplot as plt
from cartopy import  crs as ccrs

# Set up a simple integrator
def rectangle_integrator(n: int, a: float, b: float, fun: callable) -> float:
    """
    Computes the integral of fun(x) from a to b using the rectangle rule 
    with n steps via NumPy vectorization.
    """
    dx = (b - a) / n    
    x = np.linspace(a, b, n, endpoint=False)    
    return np.sum(fun(x)) * dx

# Apply to sin(x)
n_points = 100
I = rectangle_integrator(n_points, 0, np.pi, np.sin) 

# Print value and relative error
print(f'Numerical value = {I:.6f}')
print(f'Relative error  = {np.abs(I - 2) / 2:.6e}')

# Check the convergence of the method 
powers = np.arange(1, 17)
n_values = 2**powers 

errors = np.array([
    np.abs(rectangle_integrator(n, 0, np.pi, np.sin) - 2) / 2 
    for n in n_values
])

theoretical_error = 1 / n_values**2

# Plot the errors
fig, ax = plt.subplots()

ax.loglog(n_values, errors, 'k', label='Numerical Error')
ax.loglog(n_values, theoretical_error, 'k--', label=r'Theoretical $1/N^2$')

ax.set_xlabel('Number of quadrature points ($N$)')
ax.set_ylabel('Relative Error')
ax.legend()
ax.grid(True, which="both", ls="-", alpha=0.3) 

plt.show()

The rectangle rule is very simple and it can sometimes be useful. Often, however, it is worth using more complicated quadrature schemes. Here there can be an added cost in terms understanding and implementing the scheme, but they result in methods that gives better results for the same computational cost. Indeed, for the most part we can use pre-built schemes, and hence there is no need to understand all the technical details ourselves.

A powerful set of quadrature schemes are those based on the theory of orthogonal polynomials. We will not discuss in any detail how or why these work, but simply illustrate one such method in practice. The 
**Gauss-Legendre quadrature** scheme is used for integrals of the form
\begin{equation}
I = \int_{-1}^{1}f(x) \,\mathrm{d} x, 
\end{equation}
but it can be readily applied over different intervals by performing a suitable change of variables. The 
code below applies this method to again evaluate the integral of $\sin x$ over $[0,\pi]$, with the necessary points and weights being obtained using a scipy function. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import roots_legendre

# Gauss-Legendre Integrator
def gauss_legendre_integrator(n: int, a: float, b: float, fun: callable) -> float:
    """
    Computes the integral of fun(x) from a to b using 
    n-point Gauss-Legendre quadrature.
    """
    # Get nodes (x) and weights (w) defined on [-1, 1]
    x, w = roots_legendre(n)
    
    # Transformation to range [a, b]   
    factor = 0.5 * (b - a)
    x_mapped = a + factor * (x + 1)
    w_mapped = w * factor
    
    return np.sum(fun(x_mapped) * w_mapped)

# Convergence Analysis
powers = np.arange(1, 6) 
n_values = 2**powers

errors = []
theoretical_sq = []

for n in n_values:
    val = gauss_legendre_integrator(n, 0, np.pi, np.sin)
    
    err = np.abs(val - 2) / 2
    errors.append(err)
        
    theoretical_sq.append(1 / n**2)

errors = np.array(errors)
theoretical_sq = np.array(theoretical_sq)

# Plot the errors
fig, ax = plt.subplots()

ax.loglog(n_values, errors, 'ko-', label='Gauss-Legendre Error')

ax.loglog(n_values, theoretical_sq, 'k--', label=r'Rectangle Rule Scaling ($1/N^2$)')

ax.set_xlabel('Number of quadrature points ($N$)')
ax.set_ylabel('Relative Error')
ax.set_title('Convergence Comparison')
ax.legend()
ax.grid(True, which="both", ls="-", alpha=0.3)

plt.show()

Here we see a very dramatic reduction of the error using the same number of quadrature points, and hence function evaluations, with the Gauss-Legendre method being essentially exact when using only 8 points! In fact, it can be shown that a GL-quadrature with $n$ points gives exact results (up to floating-point rounding errors) for any polynomial function of order less than or equal to $2n-1$.

## Spherical polar co-ordinates

The Earth and most other planets are close to being **spherically symmetric**, which is to say that their shape is that of a sphere, while density varies only with radius. For this reason it is very useful to work with spherical polar co-ordinates $(r,\theta,\phi)$ which are related to standard Cartesian ones by:
\begin{equation}
x = r \sin \theta \cos \phi, \\
y = r \sin \theta \sin \phi, \\
z = r \cos \theta.
\end{equation}
We note that the **colatitude** $\theta$ and **longitude** $\phi$ are defined in the intervals
$[0,\pi]$ and $[0,2\pi]$, respectively.


In Cartesian co-ordinates the Laplacian operator is given by
\begin{equation}
\nabla^{2} = \frac{\partial^{2}}{\partial x^{2}} + \frac{\partial^{2}}{\partial y^{2}} + \frac{\partial^{2}}{\partial z^{2}}. 
\end{equation}
Transforming this expression to spherical polar co-ordinates is a tedious but routine exercise and leads to 
\begin{equation}
\nabla^{2} = \frac{1}{r^{2}}\frac{\partial}{\partial r}\left(r^{2}\frac{\partial}{\partial r}\,
\right) + \frac{1}{r^{2}\sin\theta}\frac{\partial}{\partial \theta}\left(
\sin\theta \frac{\partial }{\partial \theta}\,
\right) + \frac{1}{r^{2}\sin^{2}\theta}\frac{\partial^{2}}{\partial \phi^{2}}.
\end{equation}
It will be convenient to define the **Laplace-Beltrami operator** as
\begin{equation}
\nabla^{2}_{1} = \frac{1}{\sin\theta}\frac{\partial}{\partial \theta}\left(
\sin\theta \frac{\partial }{\partial \theta}\,
\right) + \frac{1}{\sin^{2}\theta}\frac{\partial^{2}}{\partial \phi^{2}}, 
\end{equation}
and hence express the Laplacian in the form
\begin{equation}
\nabla^{2} = \frac{1}{r^{2}}\frac{\partial}{\partial r}\left(r^{2}\frac{\partial}{\partial r}\,
\right) + \frac{1}{r^{2}} \nabla^{2}_{1}. 
\end{equation}
The Laplace-Beltrami operator can be seen as the Laplacian defined on the surface of a unit sphere, 
and plays an important role within the definition of spherical harmonics.

We also note that relative to spherical polar co-ordinates the volume element $\mathrm{d} V$ takes the form
\begin{equation}
\mathrm{d} V = r^{2} \sin\theta \, \mathrm{d}r \, \mathrm{d}\theta \,\mathrm{d}\phi, 
\end{equation}
while the surface element on a sphere of radius $r$ is
\begin{equation}
\mathrm{d} S = r^{2} \sin\theta \, \mathrm{d}\theta \,\mathrm{d}\phi.
\end{equation}

**You should be familiar with the definition of spherical polar co-ordinates. But you do not need to memorise any of the the formula nor be able to derive them.**

## Spherical harmonics and expansions


The spherical harmonic functions are defined through solution of the eigenvalue problem
\begin{equation}
\nabla_{1}^{2} Y_{lm} = -l(l+1)Y_{lm}.
\end{equation}
Each spherical harmonic, $Y_{lm}$, is characterised by two **integers**, its **degree** $l$ and **order** $m$. The degree
takes only non-negative values, while for each fixed degree $l$ there are $2l+1$ linearly independent spherical harmonics  with the order $-l \le m \le l$.

There are different normalisation conventions for spherical harmonics, but the one we will use is:
\begin{equation}
    \int_{\mathbb{S}^{2}} Y_{lm}\,Y_{l'm'} \,\mathrm{d} S = \delta_{ll'}\delta_{mm'}, 
\end{equation}
where $\delta_{ij}$ is the Kronecker delta symbol that equals $1$
if $i = j$ and vanishes otherwise. Such spherically harmonics are said to be **orthonormalised**.
Note that we are make use of **real-valued** spherical harmonics
as this is most convenient for the applications considered geophysical applications. There 
are also complex-valued spherical harmonics that can be useful too. The difference just comes 
down to whether the azimuthal dependence is expressed in terms of trigonometric 
functions, or as complex exponentials. 

A key property of spherical harmonics is that they can be used to expand an arbitrary function
on the unit sphere. For example, if $f:\mathbb{S}^{2} \rightarrow \mathbb{R}$ is such a function
we can write
\begin{equation}
f(\theta,\phi) = \sum_{l=0}^{\infty}\sum_{m=-l}^{l} f_{lm}\, Y_{lm}(\theta,\phi),
\end{equation}
for suitable expansion coefficients, $f_{lm}$. In fact, using the orthogonality property
of the spherical harmonics, it is clear that
\begin{equation}
f_{lm} = \int_{\mathbb{S}^{2}} f\, Y_{lm} \, \mathrm{d} S = \int_{0}^{2\pi} \int_{0}^{\pi}
f(\theta,\phi) \, Y_{lm}(\theta,\phi)\, \sin \theta \,\mathrm{d} \theta \, \mathrm{d} \phi.
\end{equation}
It follows that given a function, $f$, we can obtain its spherical harmonic expansion coefficients
by performing suitable double integrals. Conversely, the function can be recovered from these 
coefficients by summing the appropriate series.



## Working with PYSHTOOLS

To work with spherical harmonics in python, it is useful to import the module `pyshtools`. This is contains a range of useful functions to work with spherical harmonics, including fast spherical harmonic transformations that we discuss further below. It should be added that the core of pyshtools is not written in python but calls underlying Fortran routines. This allows the methods to be very substantially faster, with the situation being similar to a lot of more common python modules such as numpy where the core linear algebra routines are again written in lower-level languages.

As a first example, we can use this module to visualise spherical harmonics of different degrees and orders. In doing this some of the syntax and functionality of this library can be seen. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyshtools as pysh
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# 1. Setup Grid
lmax = 64
grid = pysh.SHGrid.from_zeros(lmax, extend=True)

# Set target degree/order
l_plot = 2
m_plot = 1

if abs(m_plot) > l_plot:
    raise ValueError("The order cannot be larger than the degree")

# Evaluate the function on the grid
for i, lat in enumerate(grid.lats()):    
    colat = 90.0 - lat
    
    for j, lon in enumerate(grid.lons()):        
        val = pysh.expand.spharm_lm(l_plot, m_plot, colat, lon, normalization='ortho')     
        grid.data[i, j] = val

# Prepare data for Cartopy
data = grid.data
lats = grid.lats()
lons = grid.lons()

#  Plotting
fig = plt.figure(figsize=(12, 8))

# Use Robinson projection for a nice global view
ax = plt.axes(projection=ccrs.Robinson())
ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)
contour = ax.contourf(
    lons, lats, data,
    transform=ccrs.PlateCarree(),
    cmap='RdBu_r', 
    levels=15
)

plt.colorbar(contour, ax=ax, orientation='horizontal', shrink=0.8, pad=0.05, label='Amplitude')
plt.title(f'Spherical Harmonic $Y_{{{l_plot}{m_plot}}}$')

plt.show()

By plotting the spherical harmonics for a range of $l$ and $m$ values, we can make the following qualitative conclusions:

 - The degree controls the dominant wavelength of the spherical harmonic, with higher $l$ meaning shorter wavelength. In fact, the following approximate relation can be established:
 \begin{equation}
 \lambda = \frac{2\pi}{l+\frac{1}{2}},
 \end{equation}
 for the non-dimensional wavelength associated with a spherical harmonic of degree $l$.
 - For a fixed $l$, the order $m$ is related to the longitudinal wavelength. 
 
The key point here is that is that if we know the length-scales contained in a given function, we can determine an appropriate **truncation degree** $L$ to use in spherical harmonic expansions. 

## Fast spherical harmonic expansions

We have seen that the $(l,m)$-th spherical harmonic expansion coefficient of a function, $f$, is given by
\begin{equation}
f_{lm}  = \int_{0}^{2\pi} \int_{0}^{\pi}
f(\theta,\phi) \, Y_{lm}(\theta,\phi)\, \sin \theta \,\mathrm{d} \theta \, \mathrm{d} \phi.
\end{equation}
Here each of the integrals can be performed using a suitable quadrature scheme. Within a spatial 
discretisation of the function based on a grid with truncation degree $L$, the number of points in both 
latitude and longitude are proportional to $L$ (the precise value depending on the details of the 
algorithms we will not consider). It is, therefore, natural to use quadrature schemes with the 
same numbers of points, and hence the naive cost of performing the double integral scales like 
$L^{2}$. If we wish to obtain all the expansion coefficients up to the trunction degree, then the 
total number of such integrals is equal to the following summation 
\begin{equation}
\sum_{l=0}^{L} (2l+1) = (L+1)^2.
\end{equation}
It follows that the cost of a spherical harmonic transform using this method scales like $L^{4}$. Such an increase of cost with truncation degree is quite high, and so this is a potentially costly method. 

Happily, there is a better way. The idea, in outline, is to recognise that the longitudinal dependence of spherical harmonics of order $m$ goes like $\sin m \phi$ or $\cos m \phi$. As a result, 
the longitudinal integrals that are required look much like those involved in Fourier transforms. From this it can be shown that **Fast Fourier Transforms** (FFTs) can be applied to simplify the calculations. The end result is a cost that scales like $L^{3}$. While this may not seem like a hugh difference, it really does help in practice.  In fact, there is a more complicated algorithm due to Driscoll \& Healy (1994) that exploits FFTs for both the longitudinal and co-latitudinal integrals
and whose cost  scales like  $(L\log L)^{2}$. Both of these fast algorithms are implemented in the `pyshtools` library.
 

For this course you do not need to know the details of fast spherical harmonic transformations. All you need to know is that they are based on numerical quadrature schemes and that they are made more efficient by the application of the FFT algorthim. Transformations done up to a truncation degree $L$ are exact 
(up to floating point errors) so long as the functions are suitably **bandlimited**, this meaning that all spherical harmonic coefficients 
above the truncation degree are zero. In practice, this condition need not be met exactly, but so long as there is not substantial power abouve the truncation degree the methods remain applicable. 

The code block below illustrates the use of these fast transformations within `pyshtools`. Using the `pyslfp` library, we load in the 
present day thickness of the Earth's ice sheets. We perform a spherical harmonic transformation and dicuss the manner in which the individual 
coefficients can be accessed. With this  explained, we truncate the coefficients at a lower degree and transform back to get a smoothed version 
of the ice sheet thickness that we can plot. 

In [ ]:
# Read in the ice sheet thickness
lmax = 256
fp = sl.FingerPrint(lmax=lmax)
fp.set_state_from_ice_ng()
ice_thickness = fp.ice_thickness

# Plot the ice thickness
fig1, ax1, im1 = sl.plot(
    ice_thickness, 
    figsize = (12,8),  
    cmap="Blues",
    colorbar_label="Ice thickness (m)",     
)


# Get the spherical harmonic coefficients -- by default pyshtools 
# doesn't used orthonormalised coefficients, but we can set this 
# as an optional argument in the transform method. 
ice_thickness_coefficients = ice_thickness.expand(normalization="ortho")

# The individual coefficients are stored within the attribute .coeffs
# which is a numpy array of shape (2, lmax+1, lmax+1). To get the 
# (l,m)-th coefficient we proceed as follows:
l = 2
m = 1
ice_thickness_lm = ice_thickness_coefficients.coeffs[1 if m < 0 else 0, l, abs(m)]
print(f'The ({l},{m}) coefficient of ice thickness is {ice_thickness_lm:.3f}m')

# Apply a degree-dependent smoothing 
lmax_smoothed = 32
for l in range(lmax+1):
    # Define a smooth filter to remove higher degrees
    filter = 1 if l <= lmax_smoothed else np.exp(-(l-lmax_smoothed)**2)
    ice_thickness_coefficients.coeffs[:, l, :] *= filter

# Do the inverse transformation
smoothed_ice_thickness = ice_thickness_coefficients.expand()

# Plot the smoothed topography
fig2, ax2, im2 = sl.plot(
    smoothed_ice_thickness, 
    figsize = (12,8),     
    cmap="Blues",
    colorbar_label="Smoothed ice thickness (m)"
)

plt.show()

## Calculating the gravity anomaly due to the surface topography

Having assembled the basic tools required, we can now see how spherical harmonic methods can be used to solve a realistic geophysical problem: how to calculate the gravity potential associated with the Earth's ice sheets. To do this we need to solve Laplace's equation
\begin{equation}
\nabla^{2}\varphi = 0, \quad \mathbf{x} \in \mathbb{R}^{3}, 
\end{equation}
subject to the jump conditions
\begin{equation}
    [\varphi]_{-}^{+} = 0, \quad [\hat{\mathbf{n}}\cdot \nabla \varphi]_{-}^{+} =  4\pi G \rho_{i} I, 
\end{equation}
across the planet's spherical reference surface $r = b$. Here $\varphi$ is the perturbation to the gravitational potential, $G$ the gravitational constant, $\rho_{i}$ the ice density, $I$ the ice sheet thickness, and $\hat{\mathbf{n}}$ the unit normal vector to the reference surface.  It is worth noting that this equation is not exact, but accurate to first order in the dimensionless quantity $I/b$. For the Earth, this quantity is much less than one, and hence the approximate theory works well.

The key idea in solving this problem is to expand the gravitational potential in spherical harmonics
\begin{equation}
\varphi(r,\theta,\phi) = \sum_{lm} \varphi_{lm}(r) \,Y_{lm}(\theta,\phi), 
\end{equation}
where we note that the radial dependence of the field is reflected through the expansion coefficients being 
functions of $r$. Using the spherical polar expression for the Laplacian
\begin{equation}
\nabla^{2} = \frac{1}{r^{2}}\frac{\partial}{\partial r}\left(r^{2}\frac{\partial}{\partial r}\,
\right) + \frac{1}{r^{2}} \nabla^{2}_{1}. 
\end{equation}
along with the properties of the spherical harmonics it can be shown that the problem reduces to solution of
\begin{equation}
 \frac{1}{r^{2}}\frac{\mathrm{d}}{ \mathrm{d} r}\left(r^{2}
\frac{\mathrm{d} \varphi_{lm}}{\mathrm{d} r}\,
\right) - \frac{l(l+1)}{r^{2}} \varphi_{lm}  = 0, 
\end{equation}
for each degree $l$ and order $m$, with these  equations being subject at $r = b$ to the 
jump conditions
\begin{equation}
[\varphi_{lm}]_{-}^{+} = 0, \quad \left[
\frac{\mathrm{d} \varphi_{lm}}{\mathrm{d} r}
\right]_{-}^{+} =  4 \pi G \rho_{i} I_{lm}, 
\end{equation}
where $I_{lm}$ are the  expansion coefficients on the topography. It is possible to solve these
ordinary differential equations in closed-form to obtain
\begin{equation}
    \varphi_{lm}(r) =  \frac{ -4\pi G \rho_{i} b\, I_{lm}}{2l+1} \left\{
    \begin{array}{ll}
     \left(
    \frac{r}{b}
    \right)^{l} & 0\le r \le b \\
     \left(
    \frac{b}{r}
    \right)^{l+1} & r \ge b
    \end{array}
    \right.. 
\end{equation}
When visualising gravitational potential perturbations on the reference surface $r = b$, it is common to convert them into the associated **geoid height anomaly** as defined by
\begin{equation}
h(\theta,\phi) = - \frac{\varphi(b,\theta,\phi)}{g}. 
\end{equation}
where $g$ is the average gravitational acceleration at the Earth's surface. 


This method is implemented within the code below:

In [ ]:
# Get some of the necessary physical parameters
rho_i = fp.ice_density
G = fp.gravitational_constant
b = fp.mean_sea_floor_radius
g = fp.gravitational_acceleration

# Set the radius at which the potential is evaluated
r = 1.0 * b

# Form the coefficients
ice_thickness_coefficients = ice_thickness.expand(normalization="ortho")

# Apply degree dependent scaling
for l in range(lmax+1):
    factor = -4 * np.pi * G * rho_i * b * (b/r)**(l+1) / (2 * l + 1)    
    ice_thickness_coefficients.coeffs[:, l, :] *= factor

# Transform back to form the potential
gravitational_potential = ice_thickness_coefficients.expand()

# Plot the results
fig, ax, im = sl.plot(
    -1 * gravitational_potential / g, 
    figsize = (12,8), 
    colorbar_label="Geoid anomaly (m)"
)
plt.show()

## What you need to know and be able to do

- Understand the purpose of numerical integration and the general form of a quadrature scheme. You should be able to explain the rectangle rule and how it can be implemented, but do not need to know the details of more complicated schemes. 

- Be familiar with the definitions of spherical polar co-ordinates, though you do not need to memorise the associated formulae. 

- Know that spherical harmonics are characterised by their degree and order, and the link between the degree and the typical length-scale. You do not need to memorise any of the formulae related to the definition of spherical harmonics, but might be asked to perform calculations given the required information. 

- Know that a function on the unit sphere can be expanded in spherical harmonics, and how the expansions coefficients can be obtained.
    
- Given appropriate formula, describe in outline the steps by which a model of topography can be converted into the associated potential or gravity anomaly. You may also   be asked to manipulate such formulae in simple ways so as to obtain physical insight. 

## Practical Exercise 1: Building a Surface Integrator

A double integral over a rectangle $[a, b] \times [c, d]$ is defined as:
$$
I = \int_{c}^{d} \int_{a}^{b} f(x, y) \, \mathrm{d}x \, \mathrm{d}y
$$
Such integrals can be approximated by repeatedly applying a 1D quadrature scheme. To do this, 
we first define 
$$
g(y) = \int_{a}^{b} f(x, y) \, \mathrm{d}x, 
$$
such that 
$$
I = \int_{c}^{d} g(y) \, \mathrm{d}y.
$$
Given a 1D quadrature scheme, we can evaluate $g(y)$ for any given $y$. this function can 
be passed to the same quadrature scheme to determine the necessary double integral. 

Implement this technique, and test it on a simple function for which you can evaluate the integral by hand.  

### Solution

A solution to the problem is shown below. For the 1D quadrature, we use the Gauss-Legendre scheme. Note that in the 2D quadrature, we
use `numpy.vectorize` to allow our function `g(y)` to be evaluated for vector arguments. 

In [ ]:
import numpy as np

def gauss_legendre_1d(n, a, b, fun):
    """
    Computes a 1D integral using n-point Gauss-Legendre quadrature.
    """
    # Get nodes (x) and weights (w) defined on [-1, 1]
    x, w = roots_legendre(n)
    
    # Transformation to range [a, b]
    factor = 0.5 * (b - a)
    x_mapped = a + factor * (x + 1)
    w_mapped = w * factor
    
    # In GL, we multiply weights by function values and sum
    return np.sum(fun(x_mapped) * w_mapped)


def integrate_2d(nx, ny, ax, bx, ay, by, f):
    """
    Computes the double integral of f(x, y) over [ax, bx] x [ay, by] 
    by repeatedly applying a 1D quadrature scheme.
    """
    
    def g(y):
        return gauss_legendre_1d(n, ax, bx, lambda x : f(x,y))
    
    # Evaluate the outer integral: I = integral of g(y) over y
    return gauss_legendre_1d(ny, ay, by, np.vectorize(g))


# Test function
func = lambda x, y: x * y

# Compute numerically
n_points = 10
numerical_result = integrate_2d(n_points, n_points, 0, 1, 0, 1, func)

print(f"Numerical result: {numerical_result:.6f}")
print(f"Analytical result: 0.250000")
print(f"Relative Error: {np.abs(numerical_result - 0.25)/0.25:.2e}")

## Practical Exercise 2: Spectral analysis

While maps provide a great spatial view of where the ice is, geophysicists often need to know which **scales** dominate the signal. Is the gravitational effect driven by massive, continent-sized features (low degree $l$), or is it dominated by smaller, localized glaciers (high degree $l$)? To answer this, we use a **power spectrum analysis**.

Your task is to calculate and plot the power per degree $l$ for the Earth's ice sheets. This will help you visualize how much "power" exists at different physical wavelengths. At degree, $l$, the power is defined by 
$$
P_{l} = \sum_{m=-l}^{l} I_{lm}^{2}, 
$$
where $I_{lm}$ is the $(l,m)$-th coefficient of the ice thickness. 

You can either determine the power directly through a loop, or you can use the method `.spectrum()` for the `SHCoeffs` class that returns 
a vector of the powers at each degree. Plot the results on a log-log plot using `plt.loglog()`.

### Solution

Using the `.spectrum()` method, the necessary code is shown below:

In [ ]:
# Calculate the power spectrum
spectrum = ice_thickness_coefficients.spectrum()
degrees = np.arange(len(spectrum))

fig, ax = plt.subplots(figsize=(10, 6))

# Plot the primary data (Degree l)
ax.loglog(degrees[1:], spectrum[1:], color='royalblue', lw=2) 
ax.set_xlabel('Spherical Harmonic Degree ($l$)')
ax.set_ylabel('Power')
ax.set_title('Power Spectrum of Ice Sheet Thickness')

ax.grid(True, which="both", ls="-", alpha=0.2)
plt.show()

## Practical Exercise 3: Calculating gravity anomalies

The gravitational anomaly, $\Delta g$, associated with the gravitational potential perturbation $\varphi$ is defined by 
$$
\Delta g = \left.\frac{\partial \varphi}{\partial r}\right|_{r=b}, 
$$
where the derivative is taken using the solution valid for $r \ge b$. Using the expression for $\varphi$ stated within the lectures, 
determine the spherical harmonic coefficients of $\Delta g$, and use this to calculate and plot the gravity anomaly.

Show that for an ice sheet that is of sufficiently small size, the associated gravity anomaly can be approximated by 
$$
\Delta g = 2\pi G \rho_{i} I, 
$$
this being a particular case of the **Bouger anomaly formula** some of you will have seen previously. 



### Solution

The gravity anomaly coefficients are:
$$
\Delta g_{lm} = \frac{4 \pi G \rho_{i} (l+1)}{2l+1}I_{lm}.
$$
This is implemented in the code below:

In [ ]:
# Set the radius at which the gravity anomaly is evaluated
r = 1.0 * b

# Form the coefficients
ice_thickness_coefficients = ice_thickness.expand(normalization="ortho")

# Apply degree dependent scaling
for l in range(lmax+1):
    factor = 4 * np.pi * G * rho_i * (l+1) / (2 * l + 1)    
    ice_thickness_coefficients.coeffs[:, l, :] *= factor

# Transform back to form the potential
gravity_anomaly = ice_thickness_coefficients.expand()

# Plot the results
fig, ax, im = sl.plot(
    gravity_anomaly, 
    figsize = (12,8), 
    colorbar_label="Gravity anomaly (m/s^2)"
)
plt.show()

A small ice sheet will have a thickness dominated by high degrees. In such cases we have approximately
$$
\Delta g_{lm} = \frac{4 \pi G \rho_{i} (l+1)}{2l+1}I_{lm} \approx 2 \pi G \rho_{i} I_{lm}, 
$$
and hence
$$
\Delta g \approx 2\pi G \rho_{i} I, 
$$
as required. 